In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import seaborn as sn
import os
from sklearn.preprocessing import OneHotEncoder
import nltk

# Extracting recipes from csv

In [2]:
#users = pd.read_csv('../Data/PP_users.csv')
#recipes = pd.read_csv('../Data/PP_recipes.csv')
raw_interactions = pd.read_csv('./RAW_interactions.csv')
raw_recipes = pd.read_csv('./RAW_recipes.csv')

In [3]:
print(raw_interactions.shape)
raw_interactions.head()

(1132367, 5)


,user_id,recipe_id,date,rating,review
0,38094,40893,2003-02-17,4,Great with a salad. Cooked on top of stove for...
1,1293707,40893,2011-12-21,5,"So simple, so delicious! Great for chilly fall..."
2,8937,44394,2002-12-01,4,This worked very well and is EASY. I used not...
3,126440,85009,2010-02-27,5,I made the Mexican topping and took it to bunk...
4,57222,85009,2011-10-01,5,"Made the cheddar bacon topping, adding a sprin..."


In [4]:
print(raw_recipes.shape)
raw_recipes.head()

(231637, 12)


,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6
2,all in the kitchen chili,112140,130,196586,2005-02-25,"['time-to-make', 'course', 'preparation', 'mai...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13
3,alouette potatoes,59389,45,68585,2003-04-14,"['60-minutes-or-less', 'time-to-make', 'course...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11
4,amish tomato ketchup for canning,44061,190,41706,2002-10-25,"['weeknight', 'time-to-make', 'course', 'main-...","[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",8


In [5]:
# merge recipes and interactions tables

dfinal = raw_interactions.merge(raw_recipes, how='inner', left_on='recipe_id', right_on='id')
print(dfinal.shape)
dfinal.head()

(1132367, 17)


,user_id,recipe_id,date,rating,review,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
0,38094,40893,2003-02-17,4,Great with a salad. Cooked on top of stove for...,white bean green chile pepper soup,40893,495,1533,2002-09-21,"['weeknight', 'time-to-make', 'course', 'main-...","[204.8, 5.0, 9.0, 26.0, 24.0, 2.0, 10.0]",4,"['combine beans , onion , chilies , 1 / 2 teas...",easy soup for the crockpot.,"['great northern beans', 'yellow onion', 'dice...",9
1,1293707,40893,2011-12-21,5,"So simple, so delicious! Great for chilly fall...",white bean green chile pepper soup,40893,495,1533,2002-09-21,"['weeknight', 'time-to-make', 'course', 'main-...","[204.8, 5.0, 9.0, 26.0, 24.0, 2.0, 10.0]",4,"['combine beans , onion , chilies , 1 / 2 teas...",easy soup for the crockpot.,"['great northern beans', 'yellow onion', 'dice...",9
2,8937,44394,2002-12-01,4,This worked very well and is EASY. I used not...,devilicious cookie cake delights,44394,20,56824,2002-10-27,"['30-minutes-or-less', 'time-to-make', 'course...","[132.3, 11.0, 39.0, 5.0, 4.0, 11.0, 5.0]",5,"['blend together cake mix , oil and eggs', 'ad...",NaN,"[""devil's food cake mix"", 'vegetable oil', 'eg...",4
3,126440,85009,2010-02-27,5,I made the Mexican topping and took it to bunk...,baked potato toppings,85009,10,64342,2004-02-25,"['15-minutes-or-less', 'time-to-make', 'course...","[2786.2, 342.0, 134.0, 290.0, 161.0, 301.0, 42.0]",3,['pick whichever topping you want to use and c...,these toppings sure makes a nice change from p...,"['mayonnaise', 'salsa', 'cheddar cheese', 'ref...",13
4,57222,85009,2011-10-01,5,"Made the cheddar bacon topping, adding a sprin...",baked potato toppings,85009,10,64342,2004-02-25,"['15-minutes-or-less', 'time-to-make', 'course...","[2786.2, 342.0, 134.0, 290.0, 161.0, 301.0, 42.0]",3,['pick whichever topping you want to use and c...,these toppings sure makes a nice change from p...,"['mayonnaise', 'salsa', 'cheddar cheese', 'ref...",13


### Filtering data
- User that has reviewed 500 <= reviews <= 1000 reviews

In [6]:
dfinal = dfinal.groupby("user_id").filter(lambda x: 500 <= len(x) <= 1000)

In [7]:
dfinal.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 97648 entries, 5 to 1132360
Data columns (total 17 columns):
user_id           97648 non-null int64
recipe_id         97648 non-null int64
date              97648 non-null object
rating            97648 non-null int64
review            97648 non-null object
name              97648 non-null object
id                97648 non-null int64
minutes           97648 non-null int64
contributor_id    97648 non-null int64
submitted         97648 non-null object
tags              97648 non-null object
nutrition         97648 non-null object
n_steps           97648 non-null int64
steps             97648 non-null object
description       96008 non-null object
ingredients       97648 non-null object
n_ingredients     97648 non-null int64
dtypes: int64(8), object(9)
memory usage: 13.4+ MB


In [8]:
dfinal.drop(columns=['id','submitted'],inplace=True)
print(dfinal.shape)
dfinal.head()

(97648, 15)


,user_id,recipe_id,date,rating,review,name,minutes,contributor_id,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
5,52282,120345,2005-05-21,4,very very sweet. after i waited the 2 days i b...,sugared raspberries,10,37449,"['15-minutes-or-less', 'time-to-make', 'course...","[838.0, 1.0, 820.0, 0.0, 2.0, 0.0, 71.0]",6,"['carefully pick over the berries , removing l...",here's an old method for preserving fruit with...,"['raspberries', 'granulated sugar']",2
16,68960,200236,2006-12-19,4,We really enjoyed this and so did both my youn...,lamb stew with tomatoes chickpeas and spices,150,398869,"['time-to-make', 'course', 'main-ingredient', ...","[606.5, 65.0, 12.0, 34.0, 65.0, 83.0, 7.0]",14,"['heat oven to 250', 'toss lamb with salt and ...",north african spices with a basic meat stew re...,"['lamb shoulder', 'salt', 'ground black pepper...",16
21,353579,225241,2007-08-14,5,"Very good, made with ingredients I usually hav...",chicken tamale pie for 2 ww core,45,67656,"['60-minutes-or-less', 'time-to-make', 'course...","[248.0, 4.0, 28.0, 21.0, 44.0, 3.0, 11.0]",17,"['pre-heat oven to 350 degrees', 'prepare a no...",after wondering how recipe #224699 would work ...,"['cooking spray', 'onion', 'green peppers', 'g...",16
35,446143,79222,2008-02-01,4,"Oh, how wonderful! I doubled the crab, and ad...",potato crab chowder,45,115419,"['60-minutes-or-less', 'time-to-make', 'course...","[274.2, 12.0, 15.0, 18.0, 32.0, 22.0, 12.0]",8,['saute onion& garlic in melted butter in larg...,soup for the soul!,"['butter', 'onion', 'garlic', 'potatoes', 'flo...",13
58,539686,321038,2011-03-28,5,My very health conscious daughter and I triple...,hg s grab n go breakfast cookies weight wat...,22,346383,"['30-minutes-or-less', 'time-to-make', 'course...","[182.4, 2.0, 50.0, 7.0, 11.0, 1.0, 13.0]",14,"['preheat oven to 375 degrees', 'chop raisins ...",i received a hungry girl email today and this ...,"['oats', 'whole wheat flour', 'fiber one cerea...",14


# Change date format (from str to date format)

In [9]:
from datetime import datetime

dfinal['date'] = dfinal['date'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d').date())

In [10]:
print(type(dfinal['date'].iloc[0]))

<class 'datetime.date'>


# Extract nutritional Components

- from string to respective categories
- convert them to float type

In [11]:
df_nutri = dfinal[['recipe_id', 'name','n_steps','n_ingredients','minutes','nutrition']]
df_nutri.drop_duplicates(inplace=True)
df_nutri.reset_index(drop=True,inplace=True)
df_nutri

C:\Users\Gi Han\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,recipe_id,name,n_steps,n_ingredients,minutes,nutrition
0,120345,sugared raspberries,6,2,10,"[838.0, 1.0, 820.0, 0.0, 2.0, 0.0, 71.0]"
1,200236,lamb stew with tomatoes chickpeas and spices,14,16,150,"[606.5, 65.0, 12.0, 34.0, 65.0, 83.0, 7.0]"
2,225241,chicken tamale pie for 2 ww core,17,16,45,"[248.0, 4.0, 28.0, 21.0, 44.0, 3.0, 11.0]"
3,79222,potato crab chowder,8,13,45,"[274.2, 12.0, 15.0, 18.0, 32.0, 22.0, 12.0]"
4,321038,hg s grab n go breakfast cookies weight wat...,14,14,22,"[182.4, 2.0, 50.0, 7.0, 11.0, 1.0, 13.0]"
...,...,...,...,...,...,...
65295,326158,whole wheat herb rolls,11,9,140,"[78.9, 3.0, 8.0, 4.0, 4.0, 6.0, 4.0]"
65296,40514,sun dried tomato bruschetta with goat cheese,17,5,15,"[94.6, 5.0, 4.0, 8.0, 8.0, 10.0, 3.0]"
65297,290157,mediterranean spice mix,4,6,5,"[10.9, 0.0, 0.0, 4.0, 0.0, 0.0, 0.0]"
65298,190698,strawberry balsamic vinaigrette,2,4,5,"[102.1, 15.0, 8.0, 0.0, 0.0, 6.0, 1.0]"


In [12]:
df_nutri[['calories','total fat (PDV)','sugar (PDV)','sodium (PDV)','protein (PDV)','saturated fat (PDV)','carbohydrates (PDV)']] = df_nutri.nutrition.str.split(",",expand=True)

C:\Users\Gi Han\Anaconda3\lib\site-packages\pandas\core\frame.py:3494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [13]:
df_nutri['calories'] =  df_nutri['calories'].apply(lambda x: x.replace('[','')) 
df_nutri['carbohydrates (PDV)'] =  df_nutri['carbohydrates (PDV)'].apply(lambda x: x.replace(']',''))

C:\Users\Gi Han\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\Gi Han\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [14]:
df_nutri[['calories','total fat (PDV)','sugar (PDV)','sodium (PDV)','protein (PDV)','saturated fat (PDV)','carbohydrates (PDV)']] = df_nutri[['calories','total fat (PDV)','sugar (PDV)','sodium (PDV)','protein (PDV)','saturated fat (PDV)','carbohydrates (PDV)']].astype('float')

In [15]:
print(df_nutri.shape)
df_nutri.drop(['nutrition'], axis=1,inplace=True)
df_nutri.head()

(65300, 13)


C:\Users\Gi Han\Anaconda3\lib\site-packages\pandas\core\frame.py:4102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,recipe_id,name,n_steps,n_ingredients,minutes,calories,total fat (PDV),sugar (PDV),sodium (PDV),protein (PDV),saturated fat (PDV),carbohydrates (PDV)
0,120345,sugared raspberries,6,2,10,838.0,1.0,820.0,0.0,2.0,0.0,71.0
1,200236,lamb stew with tomatoes chickpeas and spices,14,16,150,606.5,65.0,12.0,34.0,65.0,83.0,7.0
2,225241,chicken tamale pie for 2 ww core,17,16,45,248.0,4.0,28.0,21.0,44.0,3.0,11.0
3,79222,potato crab chowder,8,13,45,274.2,12.0,15.0,18.0,32.0,22.0,12.0
4,321038,hg s grab n go breakfast cookies weight wat...,14,14,22,182.4,2.0,50.0,7.0,11.0,1.0,13.0


# Combine dataset 
Merge with user information

In [16]:
dfuser = dfinal[['user_id', 'date', 'review','rating','recipe_id']]
dfuser.drop_duplicates(inplace=True)
dfuser.reset_index(drop=True,inplace=True)
dfuser

C:\Users\Gi Han\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,user_id,date,review,rating,recipe_id
0,52282,2005-05-21,very very sweet. after i waited the 2 days i b...,4,120345
1,68960,2006-12-19,We really enjoyed this and so did both my youn...,4,200236
2,353579,2007-08-14,"Very good, made with ingredients I usually hav...",5,225241
3,446143,2008-02-01,"Oh, how wonderful! I doubled the crab, and ad...",4,79222
4,539686,2011-03-28,My very health conscious daughter and I triple...,5,321038
...,...,...,...,...,...
97643,469903,2008-06-24,Simple flavors of summertime! Instead of bread...,4,40514
97644,2000431901,2017-11-25,SO good! I ate 4 of them fresh from the oven!,5,40514
97645,780172,2009-03-15,"I very seldom buy or make spice mixes, not sur...",5,290157
97646,333017,2009-06-24,"This was just OK. When I first tasted it, I fo...",3,190698


In [17]:
df_combine = df_nutri.merge(dfuser, how='inner', on='recipe_id')
df_combine

,recipe_id,name,n_steps,n_ingredients,minutes,calories,total fat (PDV),sugar (PDV),sodium (PDV),protein (PDV),saturated fat (PDV),carbohydrates (PDV),user_id,date,review,rating
0,120345,sugared raspberries,6,2,10,838.0,1.0,820.0,0.0,2.0,0.0,71.0,52282,2005-05-21,very very sweet. after i waited the 2 days i b...,4
1,200236,lamb stew with tomatoes chickpeas and spices,14,16,150,606.5,65.0,12.0,34.0,65.0,83.0,7.0,68960,2006-12-19,We really enjoyed this and so did both my youn...,4
2,225241,chicken tamale pie for 2 ww core,17,16,45,248.0,4.0,28.0,21.0,44.0,3.0,11.0,353579,2007-08-14,"Very good, made with ingredients I usually hav...",5
3,79222,potato crab chowder,8,13,45,274.2,12.0,15.0,18.0,32.0,22.0,12.0,446143,2008-02-01,"Oh, how wonderful! I doubled the crab, and ad...",4
4,321038,hg s grab n go breakfast cookies weight wat...,14,14,22,182.4,2.0,50.0,7.0,11.0,1.0,13.0,539686,2011-03-28,My very health conscious daughter and I triple...,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97643,40514,sun dried tomato bruschetta with goat cheese,17,5,15,94.6,5.0,4.0,8.0,8.0,10.0,3.0,469903,2008-06-24,Simple flavors of summertime! Instead of bread...,4
97644,40514,sun dried tomato bruschetta with goat cheese,17,5,15,94.6,5.0,4.0,8.0,8.0,10.0,3.0,2000431901,2017-11-25,SO good! I ate 4 of them fresh from the oven!,5
97645,290157,mediterranean spice mix,4,6,5,10.9,0.0,0.0,4.0,0.0,0.0,0.0,780172,2009-03-15,"I very seldom buy or make spice mixes, not sur...",5
97646,190698,strawberry balsamic vinaigrette,2,4,5,102.1,15.0,8.0,0.0,0.0,6.0,1.0,333017,2009-06-24,"This was just OK. When I first tasted it, I fo...",3


In [18]:
# sort dataframe by date
final = df_combine.sort_values(by="date")
final.drop_duplicates(inplace=True)
print(final.shape)
final.head()

(97648, 16)


,recipe_id,name,n_steps,n_ingredients,minutes,calories,total fat (PDV),sugar (PDV),sodium (PDV),protein (PDV),saturated fat (PDV),carbohydrates (PDV),user_id,date,review,rating
5898,8694,margaret lefever s wonderful baked beans,10,6,315,326.2,13.0,75.0,14.0,28.0,13.0,16.0,8629,2001-04-22,I tried this recipe because it was the first s...,2
73610,5315,icy holiday punch,5,6,1440,54.5,0.0,50.0,0.0,0.0,0.0,4.0,8629,2001-04-24,I brought this to an office holiday party and ...,4
83314,9014,cheesy potato skillet stuff,5,5,40,357.8,33.0,6.0,22.0,30.0,37.0,8.0,11297,2001-06-18,What a great quick & easy meal especially for ...,4
25197,580,garlic mushroom sauce,10,7,36,120.3,9.0,4.0,30.0,3.0,20.0,1.0,11297,2001-06-25,Easy and tasty. Really made my steak special....,4
84959,9492,chicken honey glazed,6,7,100,484.9,11.0,189.0,20.0,110.0,6.0,16.0,11297,2001-06-26,So easy and tasty. To make the clean-up even...,4


In [19]:
# Convert to csv
final.to_csv('./Dataset/final_dataset.csv', index=False)

# Train & Test Dataset

In [20]:
all_dataset = pd.read_csv('./Dataset/final_dataset.csv')
all_dataset.head()

,recipe_id,name,n_steps,n_ingredients,minutes,calories,total fat (PDV),sugar (PDV),sodium (PDV),protein (PDV),saturated fat (PDV),carbohydrates (PDV),user_id,date,review,rating
0,8694,margaret lefever s wonderful baked beans,10,6,315,326.2,13.0,75.0,14.0,28.0,13.0,16.0,8629,2001-04-22,I tried this recipe because it was the first s...,2
1,5315,icy holiday punch,5,6,1440,54.5,0.0,50.0,0.0,0.0,0.0,4.0,8629,2001-04-24,I brought this to an office holiday party and ...,4
2,9014,cheesy potato skillet stuff,5,5,40,357.8,33.0,6.0,22.0,30.0,37.0,8.0,11297,2001-06-18,What a great quick & easy meal especially for ...,4
3,580,garlic mushroom sauce,10,7,36,120.3,9.0,4.0,30.0,3.0,20.0,1.0,11297,2001-06-25,Easy and tasty. Really made my steak special....,4
4,9492,chicken honey glazed,6,7,100,484.9,11.0,189.0,20.0,110.0,6.0,16.0,11297,2001-06-26,So easy and tasty. To make the clean-up even...,4


In [21]:
import math

date_80th = all_dataset.iloc[math.floor(.8 * len(all_dataset))]['date']
print('Date of the 80th-percentile review: ', date_80th)  

i_date = all_dataset.index[all_dataset['date'] == date_80th].tolist()
print(f'Index of the first review on {date_80th}: ', i_date[0]) 

Date of the 80th-percentile review:  2010-06-27
Index of the first review on 2010-06-27:  78099


In [22]:
train_set = all_dataset.iloc[:i_date[0]]
print(train_set.shape)
train_set.tail()

(78099, 16)


,recipe_id,name,n_steps,n_ingredients,minutes,calories,total fat (PDV),sugar (PDV),sodium (PDV),protein (PDV),saturated fat (PDV),carbohydrates (PDV),user_id,date,review,rating
78094,424183,moroccan orange cake,19,9,50,852.4,50.0,311.0,30.0,26.0,26.0,42.0,621626,2010-06-26,We loved this cake. I did cut down on the amou...,5
78095,220853,chai tea from scratch,9,15,40,85.8,7.0,0.0,2.0,8.0,14.0,2.0,953275,2010-06-26,"Fabulous chai, well worth the effort. The onl...",5
78096,43227,white loaf,15,6,205,1252.3,57.0,19.0,107.0,57.0,111.0,65.0,496803,2010-06-26,Yum! An excellent bread. I made this through...,4
78097,429533,lemon filled ginger scones,8,7,45,215.3,12.0,29.0,9.0,8.0,25.0,10.0,539686,2010-06-26,I'm so glad I gave these a try...very deliciou...,5
78098,425060,sesame vegetable salad,14,15,22,123.5,14.0,17.0,14.0,6.0,6.0,3.0,239758,2010-06-26,Great flavours. I agree with other reviewers t...,4


In [23]:
test_set = all_dataset.iloc[i_date[0]:]
print(test_set.shape)
test_set.head()

(19549, 16)


,recipe_id,name,n_steps,n_ingredients,minutes,calories,total fat (PDV),sugar (PDV),sodium (PDV),protein (PDV),saturated fat (PDV),carbohydrates (PDV),user_id,date,review,rating
78099,63160,spicy chicken dippers,7,5,30,621.2,47.0,3.0,16.0,77.0,31.0,15.0,383853,2010-06-27,Awesome recipe! I've made similar ones before...,5
78100,137364,perfect basic white rice,13,5,20,168.8,0.0,0.0,24.0,6.0,0.0,12.0,178427,2010-06-27,This was very good but slightly overcooked for...,4
78101,425009,chile vinegar sauce,4,3,10,13.3,0.0,13.0,0.0,0.0,0.0,1.0,678366,2010-06-27,This reminded me of the sauce I used in Bankgo...,5
78102,425258,slow cooker pork chops with sauerkraut,4,10,315,279.4,26.0,13.0,37.0,48.0,25.0,2.0,780172,2010-06-27,This was a very nice pork meal and easy to pre...,4
78103,115270,carrots zucchini and basil,8,10,35,42.1,3.0,9.0,1.0,2.0,1.0,1.0,187373,2010-06-27,"This was a fun, colorful side dish with supper...",5


In [24]:
# convert to csv
train_set.to_csv('./Dataset/train_dataset_all.csv', index=False)
test_set.to_csv('./Dataset/test_dataset_all.csv', index=False)

## Test-validation dataset

In [25]:
val_set = test_set[:len(test_set)//2]
val_set

,recipe_id,name,n_steps,n_ingredients,minutes,calories,total fat (PDV),sugar (PDV),sodium (PDV),protein (PDV),saturated fat (PDV),carbohydrates (PDV),user_id,date,review,rating
78099,63160,spicy chicken dippers,7,5,30,621.2,47.0,3.0,16.0,77.0,31.0,15.0,383853,2010-06-27,Awesome recipe! I've made similar ones before...,5
78100,137364,perfect basic white rice,13,5,20,168.8,0.0,0.0,24.0,6.0,0.0,12.0,178427,2010-06-27,This was very good but slightly overcooked for...,4
78101,425009,chile vinegar sauce,4,3,10,13.3,0.0,13.0,0.0,0.0,0.0,1.0,678366,2010-06-27,This reminded me of the sauce I used in Bankgo...,5
78102,425258,slow cooker pork chops with sauerkraut,4,10,315,279.4,26.0,13.0,37.0,48.0,25.0,2.0,780172,2010-06-27,This was a very nice pork meal and easy to pre...,4
78103,115270,carrots zucchini and basil,8,10,35,42.1,3.0,9.0,1.0,2.0,1.0,1.0,187373,2010-06-27,"This was a fun, colorful side dish with supper...",5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87868,86086,pink powder puff chex crunch,13,6,10,133.0,6.0,66.0,4.0,2.0,10.0,8.0,1535,2012-03-08,ADDICTIVE!! I can't get over how good this is....,5
87869,46285,lemon poppy seed quick bread,10,12,70,464.5,21.0,101.0,26.0,17.0,35.0,25.0,1535,2012-03-08,I can't rate the taste yet since I sent it to ...,0
87870,13858,white castle cheeseburgers,14,6,40,176.4,12.0,5.0,12.0,21.0,19.0,4.0,65197,2012-03-08,I made like a 'zillion' of these for our Nasc...,5
87871,452308,green beans with brown butter and pecans,5,4,20,147.2,20.0,12.0,0.0,4.0,31.0,2.0,135470,2012-03-08,This is super simple and super tasty! I wasn't...,5


In [26]:
test_set2 = test_set[len(test_set)//2:]
test_set2

,recipe_id,name,n_steps,n_ingredients,minutes,calories,total fat (PDV),sugar (PDV),sodium (PDV),protein (PDV),saturated fat (PDV),carbohydrates (PDV),user_id,date,review,rating
87873,107889,beef and onion soup,16,12,82,306.5,28.0,12.0,58.0,49.0,42.0,2.0,632249,2012-03-08,I made this using leftover beef stock from rec...,4
87874,75622,golden graham chocolate chip bars,12,13,45,5395.9,428.0,1719.0,190.0,181.0,658.0,228.0,131674,2012-03-08,Big hit with the kids and me cause I got to us...,5
87875,437870,scallop pie,10,9,30,453.0,28.0,11.0,51.0,50.0,55.0,9.0,50509,2012-03-09,"I liked the flavor of this dish, but did not t...",4
87876,474386,chicken parmesan casserole,11,9,70,571.7,42.0,36.0,52.0,97.0,55.0,9.0,233583,2012-03-09,WOW!!!!!!!!!!!! ANd WOW again! I had planned...,5
87877,382575,no cook roast beef salad with goat cheese and ...,3,10,15,318.1,34.0,13.0,54.0,42.0,34.0,2.0,865936,2012-03-09,Thoroughly enjoyed this salad for dinner. I u...,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97643,140486,basic vanilla frosting,1,4,5,1162.8,37.0,940.0,9.0,2.0,76.0,80.0,227652,2018-12-05,This was incredibly simple and easy to put tog...,5
97644,121066,garlic shrimp pasta,18,10,30,408.1,14.0,8.0,20.0,41.0,25.0,19.0,198154,2018-12-09,I doubled the garlic and felt that it could us...,4
97645,19653,roll out cookies,12,6,25,1870.6,147.0,402.0,50.0,43.0,296.0,77.0,227652,2018-12-10,These cookies were wonderful. I've had this in...,5
97646,23113,pumpkin risotto,14,12,30,405.6,16.0,8.0,9.0,22.0,23.0,21.0,198154,2018-12-11,I made this as a side because DH is a meat and...,4


In [27]:
# convert to csv
val_set.to_csv('./Dataset/validation_dataset_all.csv', index=False)
test_set2.to_csv('./Dataset/test_v2_dataset_all.csv', index=False)